___

# <p style="text-align: center"> Invisible Influencers </p>
## <p style="text-align: center"> Investigating YouTube Bot's Phenomenon </p>

___

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import polars as pl
import glob
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import datetime
import os

# Ensure plotly.js is set to default
pio.kaleido.scope.plotlyjs = None

In [ ]:
# Set Default Plotting Params

font_family = "Arial"  # Replace with your desired font
plt.rcParams["font.family"] = font_family

# Optional: Set font size and weight
plt.rcParams["font.size"] = 12
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

# Apply Seaborn theme (inherits Matplotlib fonts)
sns.set_theme(style="whitegrid", font=font_family)

# Apply Plotly theme
plotly_layout = {
    "font": {
        "family": font_family,
        "size": 12,  # Match with plt.rcParams["font.size"]
    }
}

___

## <p style="text-align: center"> <span style="text-decoration: underline"> **Type-1 Bot Analysis** </span> </p>

#### - **Data Preprocessing and Loading**

In [ ]:
combine = False

if combine:

    datasets = ['normal', 'suspicious']

    for dataset in datasets:
        print(f"reading partial files '{dataset}_i.parquet'...")

        # List all Parquet files
        parquet_files = glob.glob(f'./data_type1/{dataset}_users_*.parquet')

        # Read and concatenate all Parquet files
        combined = pl.concat([pl.read_parquet(file) for file in parquet_files])
        combined.write_parquet(f'data_type1/combi_{dataset}_dataset1.parquet')

In [ ]:
# Load Data

type_1=pl.read_parquet('data/data_type1/combi_dataset1.parquet')
df_sus=pl.read_parquet('data/data_type1/combi_suspicious_dataset1.parquet')

In [ ]:
# Add a column 'year' to the dataframe

type_1 = type_1.with_columns([
    pl.col("upload_date").cast(pl.Date),  # Ensure it's in datetime format
    pl.col("upload_date").dt.year().alias("year")  # Extract year
])

df_sus = df_sus.with_columns([
    pl.col("upload_date").cast(pl.Date),  # Ensure it's in datetime format
    pl.col("upload_date").dt.year().alias("year")  # Extract year
])

df_sus

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **Do Bots Target Specific Video Categories ?** </span> </p>

In [ ]:
# Colormap for consistency

colormap_categories = {
    "Entertainment": "red",
    "Other": "orange",
    "Gaming": "cyan",
    "People & Blogs": "yellow",
    "News & Politics": "lime",
    "Howto & Style": "lightblue",
    "Music": "blue",
    "Education": "lightgreen",
    "Science & Technology": "brown",
    "Film & Animation": "pink",
    "Comedy": "green",
    "Sports": "purple",
    "Pets & Animals": "teal",
    "Travel & Events": "lavender",
    "Autos & Vehicles": "salmon",
    "Nonprofits & Activism": "gold",
    "Shows": "gold",
    "Trailers": "lightcoral"
}

order_categories = {
    "Entertainment": 1,
    "Gaming": 2,
    "People & Blogs": 3,
    "News & Politics": 4,
    "Howto & Style": 5,
    "Music": 6,
    "Education": 7,
    "Science & Technology": 8,
    "Film & Animation": 9,
    "Comedy": 10,
    "Sports": 11,
    "Pets & Animals": 12,
    "Travel & Events": 13,
    "Autos & Vehicles": 14,
    "Nonprofits & Activism": 15,
    "Shows": 16,
    "Trailers": 17,
    "Other": 18
}

#### - **Suspicious Users**

In [ ]:
# Compute the number of comments per category of videos
df_comm_per_cat = df_sus.group_by(["year","categories"]).agg(pl.col("comments").sum()).filter(pl.col("categories")!="").sort(by=["year","comments"], descending=True)

# Add proportion of comments per category per year
comments_per_year = df_comm_per_cat.group_by("year").agg(pl.col("comments").sum().alias("total_comments")).sort(by="year")
df_comm_per_cat = df_comm_per_cat.join(comments_per_year, on="year")
df_comm_per_cat = df_comm_per_cat.with_columns([
    (pl.col("comments") / pl.col("total_comments") * 100.0).alias("proportion")
])

# Keep only categories with more than 2% of the comments, put others in a category 'Other'
df_comm_per_cat = df_comm_per_cat.with_columns(pl.when(pl.col("proportion")<5.0).then(pl.lit("Other")).otherwise(pl.col("categories")).alias("Categories")).drop("categories")
df_comm_per_cat = df_comm_per_cat.group_by(["year","Categories"]).agg(pl.col("comments").sum(), pl.col("proportion").sum()).sort(by=["year","comments"], descending=True)

# Add order column for plotting
df_comm_per_cat = df_comm_per_cat.with_columns([
    pl.col("Categories").map_elements(lambda x: order_categories[x]).alias("order")
])

df_comm_per_cat = df_comm_per_cat.sort(by=["year","order"])

df_comm_per_cat

In [ ]:
fig = px.area(df_comm_per_cat.to_pandas(), x="year", y="proportion", color="Categories", 
              labels={"proportion":"Proportion of Comments (%)", "year":"Year", "Categories":"Category"},
              color_discrete_map=colormap_categories)
fig.show()
fig.write_image("./image_aurel/prop_comments_per_category_sus.svg")
fig.write_html("./image_aurel/prop_comments_per_category_sus.html")

#### - **Normal Users**

In [ ]:
# Compute the number of comments per category of videos
df_comm_per_cat_normal = type_1.group_by(["year","categories"]).agg(pl.col("comments").sum()).filter(pl.col("categories")!="").sort(by=["year","comments"], descending=True)

# Add proportion of comments per category per year
comments_per_year_normal = df_comm_per_cat_normal.group_by("year").agg(pl.col("comments").sum().alias("total_comments")).sort(by="year")
df_comm_per_cat_normal = df_comm_per_cat_normal.join(comments_per_year_normal, on="year")
df_comm_per_cat_normal = df_comm_per_cat_normal.with_columns([
    (pl.col("comments") / pl.col("total_comments") * 100.0).alias("proportion")
])

# Keep only categories with more than 4% of the comments, put others in a category 'Other'
df_comm_per_cat_normal = df_comm_per_cat_normal.with_columns(pl.when(pl.col("proportion")<5.0).then(pl.lit("Other")).otherwise(pl.col("categories")).alias("Categories")).drop("categories")
df_comm_per_cat_normal = df_comm_per_cat_normal.group_by(["year","Categories"]).agg(pl.col("comments").sum(), pl.col("proportion").sum()).sort(by=["year","comments"], descending=True)

# Add order column for plotting
df_comm_per_cat_normal = df_comm_per_cat_normal.with_columns([
    pl.col("Categories").map_elements(lambda x: order_categories[x]).alias("order")
])

df_comm_per_cat_normal = df_comm_per_cat_normal.sort(by=["year","order"])

In [ ]:
fig = px.area(df_comm_per_cat_normal.to_pandas(), x="year", y="proportion", color="Categories", 
            labels={"proportion":"Proportion of Comments (%)", "year":"Year", "Categories":"Category"},
            color_discrete_map=colormap_categories)
fig.show()


fig.write_image("./image_aurel/prop_comments_per_category_normal.svg")
fig.write_html("./image_aurel/prop_comments_per_category_normal.html")


#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **Do Bots Target One or Many Channels ?** </span> </p>

In [ ]:
# Color map for consistency

colormap_cat_channel = {
    "1": "red",
    "2": "blue",
    "3": "green",
    "4": "orange",
    "5+": "cyan",
}

order_cat_channel = {
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5+": 5,
}

#### - **Suspicious Users**

In [ ]:
df_nb_channels_per_bot = df_sus.group_by(["year","author"]).agg(pl.col("channel_id").n_unique().alias("nb_channels")).sort(by=["year","nb_channels"], descending=True)
df_nb_channels_per_bot = df_nb_channels_per_bot.with_columns(pl.when(pl.col("nb_channels")>=5).then(pl.lit("5+")).otherwise(pl.col("nb_channels")).alias("nb_channels")).group_by(["year","nb_channels"]).agg(pl.col("nb_channels").count().alias("nb_users")).sort(by=["year","nb_users"], descending=True)

# Proportion per year 

nb_bots_per_year = df_nb_channels_per_bot.group_by("year").agg(pl.col("nb_users").sum().alias("total_users")).sort(by="year")

df_nb_channels_per_bot = df_nb_channels_per_bot.join(nb_bots_per_year, on="year")
df_nb_channels_per_bot = df_nb_channels_per_bot.with_columns([
    (pl.col("nb_users") / pl.col("total_users") * 100.0).alias("proportion")
])

df_nb_channels_per_bot = df_nb_channels_per_bot.with_columns([
    pl.col("nb_channels").map_elements(lambda x: order_cat_channel[x]).alias("order")
])

df_nb_channels_per_bot = df_nb_channels_per_bot.sort(by=["year","order"])

In [ ]:
fig = px.area(df_nb_channels_per_bot.to_pandas(), x="year", y="proportion", color="nb_channels", 
              labels={"proportion":"Proportion of Bots (%)", "year":"Year", "nb_channels":"Channels Targeted"},
              color_discrete_map=colormap_cat_channel)
fig.show()


fig.write_image("./image_aurel/prop_bots_per_channels_sus.svg")
fig.write_html("./image_aurel/prop_bots_per_channels_sus.html")

#### - **Normal Users**

In [ ]:
df_nb_channels_per_user = type_1.group_by(["year","author"]).agg(pl.col("channel_id").n_unique().alias("nb_channels")).sort(by=["year","nb_channels"], descending=True)
df_nb_channels_per_user = df_nb_channels_per_user.with_columns(pl.when(pl.col("nb_channels")>=5).then(pl.lit("5+")).otherwise(pl.col("nb_channels")).alias("nb_channels")).group_by(["year","nb_channels"]).agg(pl.col("nb_channels").count().alias("nb_users")).sort(by=["year","nb_users"], descending=True)

# Proportion per year 

nb_user_per_year = df_nb_channels_per_user.group_by("year").agg(pl.col("nb_users").sum().alias("total_users")).sort(by="year")

df_nb_channels_per_user = df_nb_channels_per_user.join(nb_user_per_year, on="year")
df_nb_channels_per_user = df_nb_channels_per_user.with_columns([
    (pl.col("nb_users") / pl.col("total_users") * 100.0).alias("proportion")
])

df_nb_channels_per_user = df_nb_channels_per_user.with_columns([
    pl.col("nb_channels").map_elements(lambda x: order_cat_channel[x]).alias("order")
])

df_nb_channels_per_user = df_nb_channels_per_user.sort(by=["year","order"])

In [ ]:
fig = px.area(df_nb_channels_per_user.to_pandas(), x="year", y="proportion", color="nb_channels", 
              labels={"proportion":"Proportion of Normal users (%)", "year":"Year", "nb_channels":"Channels Targeted"},
              color_discrete_map=colormap_cat_channel)
fig.show()


fig.write_image("./image_aurel/prop_normal_per_channels.svg")
fig.write_html("./image_aurel/prop_normal_per_channels.html")

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How Different are Normal Users and Bots in Commenting Behaviors ?** </span> </p>

In [ ]:
data_type1_comm_per_user = type_1.select(["author", "comments"]).group_by("author").agg(pl.col("comments").sum()).sort(by="comments", descending=True)

df_sus_comm_per_bot= df_sus.select(["author", "comments"]).group_by("author").agg(pl.col("comments").sum()).sort(by="comments", descending=True)

df_comm_per_bot = pl.concat([data_type1_comm_per_user.with_columns([
    pl.lit("Normal").alias("Type")
]), df_sus_comm_per_bot.with_columns([
    pl.lit("Suspicious").alias("Type")
])])

In [ ]:
# box plot of comments per user for normal and suspicious users

fig = px.box(df_comm_per_bot.to_pandas(), x="Type", y="comments", color="Type",
         labels={"comments":"Number of Comments", "Type":"User Type"},
         log_y=True)
fig.update_yaxes(range=[None, 10**5])


# Save the plot
fig.write_image("./image_aurel/boxplot_comments_per_user.png")

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How Do Metrics Vary Over Time For Normal Users and Bots ?** </span> </p>

#### - **Suspicious Users**

In [ ]:
# Generate Metrics Data

# Define chunk size
chunk_size_sus = 1_000_000  # Adjust based on memory constraints

# Initialize lists to collect results
unique_users_results_sus = []
chunk_results_sus = []

# Iterate through the dataset in chunks
for start_sus in range(0, len(df_sus), chunk_size_sus):
    # Slice the DataFrame for the current chunk
    chunk_sus = df_sus[start_sus : start_sus + chunk_size_sus]

    # Fill missing values in specific columns (only if they exist)
    columns_to_fill_sus = ["comments", "likes", "replies", "view_count"]
    for col_sus in columns_to_fill_sus:
        if col_sus in chunk_sus.columns:
            chunk_sus = chunk_sus.with_columns(pl.col(col_sus).fill_null(0))

    # Aggregate metrics for this chunk
    chunk_metrics_sus = (
        chunk_sus.group_by("year")
        .agg([
            pl.col("comments").sum().alias("total_comments"),
            pl.col("likes").sum().alias("total_likes"),
            pl.col("replies").sum().alias("total_replies"),
            pl.col("view_count").sum().alias("total_views"),
            pl.col("video_id").count().alias("total_videos"),
        ])
    )
    chunk_results_sus.append(chunk_metrics_sus)

# Combine all chunk results for aggregated metrics
final_metrics_sus = pl.concat(chunk_results_sus).group_by("year").sum()

# Iterate through the dataset again for unique users
for start_sus in range(0, len(df_sus), chunk_size_sus):
    # Slice the DataFrame for the current chunk
    chunk_sus = df_sus[start_sus : start_sus + chunk_size_sus]

    # Calculate unique users per year
    unique_users_sus = (
        chunk_sus.group_by("year")
        .agg(pl.col("author").n_unique().alias("unique_users"))
    )
    unique_users_results_sus.append(unique_users_sus)

# Combine all unique users results
unique_users_combined_sus = pl.concat(unique_users_results_sus).group_by("year").sum()

# Merge the aggregated metrics with unique users
final_result_sus = final_metrics_sus.join(unique_users_combined_sus, on="year", how="left")

# Compute comments per user
final_result_sus = final_result_sus.with_columns(
    (pl.col("total_comments") / pl.col("unique_users")).alias("comments_per_user")
)
# Sort the DataFrame by year
final_result_sus = final_result_sus.sort("year")
# Display the final result
print(final_result_sus)


In [ ]:
# Normalize Metrics

final_result_sus = final_result_sus.with_columns([
    (pl.col("total_likes") / final_result_sus["total_likes"].max()).alias("Total Likes"),
    (pl.col("total_comments") / final_result_sus["total_comments"].max()).alias("Total Comments"),
    (pl.col("comments_per_user") / final_result_sus["comments_per_user"].max()).alias("Comments per User"),
    (pl.col("total_replies") / final_result_sus["total_replies"].max()).alias("Total Replies"),
])

# Remove year 2005
final_result_sus = final_result_sus.filter(pl.col("year")>2005)


colors = ["red", "blue", "green", "orange", "cyan"]

In [ ]:
fig = px.line(final_result_sus.to_pandas(), x="year", y=["Total Likes", "Total Comments", "Comments per User", "Total Replies"],
                labels={"value":"Normalized Value", "year":"Year", "variable":"Metric"},
                color_discrete_sequence=colors, markers=True)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2006+x for x in range(0,13, 2)],
        ticktext = [str(2006+x) for x in range(0,13, 2)]
    )
)
fig.show()

fig.write_image("./image_aurel/normalized_metrics_sus.svg")
fig.write_html("./image_aurel/normalized_metrics_sus.html")


#### - **Normal Users**

In [ ]:
# Generate Metrics Data

# Define chunk size
chunk_size = 1_000_000  # Adjust based on memory constraints

# Initialize lists to collect results
unique_users_results = []
chunk_results = []

# Iterate through the dataset in chunks
for start in range(0, len(type_1), chunk_size):
    # Slice the DataFrame for the current chunk
    chunk = type_1[start : start + chunk_size]

    # Fill missing values in specific columns (only if they exist)
    columns_to_fill = ["comments", "likes", "replies", "view_count"]
    for col in columns_to_fill:
        if col in chunk.columns:
            chunk = chunk.with_columns(pl.col(col).fill_null(0))

    # Aggregate metrics for this chunk
    chunk_metrics = (
        chunk.group_by("year")
        .agg([
            pl.col("comments").sum().alias("total_comments"),
            pl.col("likes").sum().alias("total_likes"),
            pl.col("replies").sum().alias("total_replies"),
            pl.col("view_count").sum().alias("total_views"),
            pl.col("video_id").count().alias("total_videos"),
        ])
    )
    chunk_results.append(chunk_metrics)

# Combine all chunk results for aggregated metrics
final_metrics = pl.concat(chunk_results).group_by("year").sum()

# Iterate through the dataset again for unique users
for start in range(0, len(type_1), chunk_size):
    # Slice the DataFrame for the current chunk
    chunk = type_1[start : start + chunk_size]

    # Calculate unique users per year
    unique_users = (
        chunk.group_by("year")
        .agg(pl.col("author").n_unique().alias("unique_users"))
    )
    unique_users_results.append(unique_users)

# Combine all unique users results
unique_users_combined = pl.concat(unique_users_results).group_by("year").sum()

# Merge the aggregated metrics with unique users
final_result = final_metrics.join(unique_users_combined, on="year", how="left")

# Compute comments per user
final_result = final_result.with_columns(
    (pl.col("total_comments") / pl.col("unique_users")).alias("comments_per_user")
)

# Display the final result
print(final_result)


In [ ]:
# Normalize Metrics

final_result = final_result.with_columns([
    (pl.col("total_likes") / final_result["total_likes"].max()).alias("Total Likes"),
    (pl.col("total_comments") / final_result["total_comments"].max()).alias("Total Comments"),
    (pl.col("comments_per_user") / final_result["comments_per_user"].max()).alias("Comments per User"),
    (pl.col("total_replies") / final_result["total_replies"].max()).alias("Total Replies"),
])

colors = ["red", "blue", "green", "orange", "cyan"]
markers = {
    "Total Likes": "circle",
    "Total Comments": "square",
    "Comments per User": "diamond",
    "Total Replies": "triangle-up",
}

In [ ]:
fig = px.line(final_result.to_pandas(), x="year", y=["Total Likes", "Total Comments", "Comments per User", "Total Replies"],
                labels={"value":"Normalized Value", "year":"Year", "variable":"Metric"},
                color_discrete_sequence=colors, markers=True)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2006+x for x in range(0,13, 2)],
        ticktext = [str(2006+x) for x in range(0,13, 2)]
    )
)
fig.show()

fig.write_image("./image_aurel/normalized_metrics.svg")
fig.write_html("./image_aurel/normalized_metrics.html")

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How different are bots and normal users?** </span> </p>

#### - **Suspicious Users**

In [ ]:
# Filter suspicious users to retain only the necessary columns
data_com_vid_year_sus= df_sus.select(["year", "author", "comments", "video_id"])

In [ ]:
# Group by year and author to compute metrics

grouped_data_sus = (
    data_com_vid_year_sus.group_by(["year", "author"])  # Group by year and author
    .agg([
        pl.col("comments").sum().alias("total_comments"),  # Total comments by user per year
        pl.col("video_id").n_unique().alias("distinct_videos_commented"),  # Unique videos commented on
    ])
    .with_columns([
        (pl.col("total_comments") / pl.col("distinct_videos_commented")).alias("avg_comments_per_user")  # Avg comments per user
    ])
)

In [ ]:
#subsample suspicious users because the plot would be impossible to make :

# Group data by year
grouped_sus = grouped_data_sus.group_by('year')

# Calculate the total subsample size (1/100th of the total rows)
total_subsample_size = len(grouped_data_sus) // 1

# Subsampling uniformly across years
subsampled_list_sus = []
for year, group in grouped_sus:
    # Ensure `group` is a DataFrame
    group = pd.DataFrame(group)
    
    # Calculate the number of samples for this year (proportional to the group's size)
    year_sample_size = max(1, len(group) * total_subsample_size // len(grouped_data_sus))
    
    # Randomly sample the data for this year
    subsampled_list_sus.append(group.sample(n=year_sample_size, random_state=42))

# Combine the sampled data from all years
subsampled_df_sus = pd.concat(subsampled_list_sus)

# Define the correct column names
column_names = ['year', 'author', 'total_comments', 'distinct_videos_commented', 'avg_comments_per_user']

# Assign the column names back to the DataFrame
subsampled_df_sus.columns = column_names

In [ ]:
# Ensure the year column is integer for proper sorting FOR SUSPICIOUS USERS
subsampled_df_sus['year'] = subsampled_df_sus['year'].astype(int)

# Sort the DataFrame by year
subsampled_df_sus = subsampled_df_sus.sort_values(by='year', ascending=True)

# Reset the index after sorting (optional)
subsampled_df_sus = subsampled_df_sus.reset_index(drop=True)

In [ ]:
# Ridge Line Plot for Suspicious Users

# Ensure Seaborn has a clean theme
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Ensure 'year' is treated as categorical for the plot
subsampled_df_sus['year'] = subsampled_df_sus['year'].astype(int)  # Convert years to integers
subsampled_df_sus['year'] = subsampled_df_sus['year'].astype(str)  # Convert to string for categorical use

# Generate a color palette for years
pal = sns.color_palette(palette='viridis', n_colors=subsampled_df_sus['year'].nunique())

# Create the FacetGrid for ridgeline
g = sns.FacetGrid(
    subsampled_df_sus,
    row='year',
    hue='year',
    aspect=15,  # Stretch plots horizontally
    height=0.5,  # Adjust height of each row
    palette=pal,
)

# Add density plots (kde)
g.map(
    sns.kdeplot,
    'avg_comments_per_user',
    bw_adjust=0.2,
    clip=(0, 50),  # Clip x-axis range to 0-5
    #clip_on=False,
    fill=True,
    alpha=1,
    linewidth=1.5,
)
# g.set_titles("")
# g.set_axis_labels("", "")
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

# Add a white contour line around each density plot
g.map(
    sns.kdeplot,
    'avg_comments_per_user',
    bw_adjust=0.2,
    clip=(0, 50),  # Clip x-axis range to 0-5
    #clip_on=False,  
    color="w",
    lw=2,
)

# Add a horizontal line at y=0 for each plot
g.map(plt.axhline, y=0, lw=2, clip_on=False)

# Add year labels to each plot
for i, ax in enumerate(g.axes.flat):
    ax.text(
        -0.5, 0.02,  # Adjust the position of the year label
        subsampled_df_sus['year'].unique()[i],
        fontweight='bold',
        fontsize=12,
        color=ax.lines[-1].get_color(),
    )

# Adjust subplot overlap
g.fig.subplots_adjust(hspace=-0.5)

# Remove the density label from each y-axis
for ax in g.axes.flat:
    ax.set_ylabel("")  # Remove the y-axis labels for density
    ax.set_xlim(0, 50)  # Set x-axis range to 0 - 50    


# Remove unnecessary axes details
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

# Set x-axis label
plt.xlabel("Average Comments per User", fontweight='bold', fontsize=12)
g.fig.suptitle('Distribution of Avg Comments per User Across Years',
               ha='right', fontsize=16, fontweight='bold')

# Show the plot
plt.show()

g.savefig("./image_aurel/ridge_line_sus.svg")
g.savefig("./image_aurel/ridge_line_sus.html")

#### - **Normal Users**

In [ ]:
# Filter normal users to retain only the necessary columns
data_com_vid_year= type_1.select(["year", "author", "comments", "video_id"])

In [ ]:
# Group by year and author to compute metrics
grouped_data = (
    data_com_vid_year.group_by(["year", "author"])  # Group by year and author
    .agg([
        pl.col("comments").sum().alias("total_comments"),  # Total comments by user per year
        pl.col("video_id").n_unique().alias("distinct_videos_commented"),  # Unique videos commented on
    ])
    .with_columns([
        (pl.col("total_comments") / pl.col("distinct_videos_commented")).alias("avg_comments_per_user")  # Avg comments per user
    ])
)

In [ ]:
#subsample normal users because the plot would be impossible to make :

# Group data by year
grouped = grouped_data.group_by('year')

# Calculate the total subsample size (1/100th of the total rows)
total_subsample_size = len(grouped_data) // 100

# Subsampling uniformly across years
subsampled_list = []
for year, group in grouped:
    # Ensure `group` is a DataFrame
    group = pd.DataFrame(group)
    
    # Calculate the number of samples for this year (proportional to the group's size)
    year_sample_size = max(1, len(group) * total_subsample_size // len(grouped_data))
    
    # Randomly sample the data for this year
    subsampled_list.append(group.sample(n=year_sample_size, random_state=42))

# Combine the sampled data from all years
subsampled_df = pd.concat(subsampled_list)

# Define the correct column names
column_names = ['year', 'author', 'total_comments', 'distinct_videos_commented', 'avg_comments_per_user']

# Assign the column names back to the DataFrame
subsampled_df.columns = column_names

In [ ]:
# Ridge Line Plot for Normal Users

# Ensure the year column is integer for proper sorting
subsampled_df['year'] = subsampled_df['year'].astype(int)

# Sort the DataFrame by year
subsampled_df = subsampled_df.sort_values(by='year', ascending=True)

# Reset the index after sorting (optional)
subsampled_df = subsampled_df.reset_index(drop=True)

# Ensure Seaborn has a clean theme
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Prepare data (already in `subsampled_df`)
# Ensure 'year' is treated as categorical for the plot
subsampled_df['year'] = subsampled_df['year'].astype(int)  # Convert years to integers
subsampled_df['year'] = subsampled_df['year'].astype(str)  # Convert to string for categorical use

# Generate a color palette for years
pal = sns.color_palette(palette='viridis', n_colors=subsampled_df['year'].nunique())

# Create the FacetGrid for ridgeline
g = sns.FacetGrid(
    subsampled_df,
    row='year',
    hue='year',
    aspect=15,  # Stretch plots horizontally
    height=0.5,  # Adjust height of each row
    palette=pal,
)

# Add density plots (kde)
g.map(
    sns.kdeplot,
    'avg_comments_per_user',
    bw_adjust=1,  # Bandwidth adjustment
    clip_on=False,
    fill=True,
    alpha=1,
    linewidth=1.5,
)
# g.set_titles("")
# g.set_axis_labels("", "")
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

# Add a white contour line around each density plot
g.map(
    sns.kdeplot,
    'avg_comments_per_user',
    bw_adjust=1,
    clip_on=False,  
    color="w",
    lw=2,
)

# Add a horizontal line at y=0 for each plot
g.map(plt.axhline, y=0, lw=2, clip_on=False)

# Add year labels to each plot
for i, ax in enumerate(g.axes.flat):
    ax.text(
        -0.5, 0.02,  # Adjust the position of the year label
        subsampled_df['year'].unique()[i],
        fontweight='bold',
        fontsize=12,
        color=ax.lines[-1].get_color(),
    )

# Adjust subplot overlap
g.fig.subplots_adjust(hspace=-0.5)

# Remove the density label from each y-axis
for ax in g.axes.flat:
    ax.set_ylabel("")  # Remove the y-axis labels for density

# Remove unnecessary axes details
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

# Set x-axis label
plt.xlabel("Average Comments per User", fontweight='bold', fontsize=12)
g.fig.suptitle('Distribution of Avg Comments per User Across Years',
               ha='right', fontsize=16, fontweight='bold')

# Show the plot
plt.show()

g.savefig("./image_aurel/ridge_line_normal.svg")

#### - **3D Plot**

In [ ]:

# Combine both datasets for SVM training
combined_df = pd.concat([
    subsampled_df.assign(label=0),  # Normal users
    subsampled_df_sus.assign(label=1)  # Suspicious users
])

# Combine features and labels
X = combined_df[["year", "avg_comments_per_user", "distinct_videos_commented"]].values
y = combined_df["label"].values

# Train the SVM
svm = SVC(kernel="linear", C=1)
svm.fit(X, y)

# Extract hyperplane parameters
w = svm.coef_[0]  # Weights
b = svm.intercept_[0]  # Intercept

# Create mesh grid for the hyperplane
x_range = np.linspace(combined_df["year"].min(), combined_df["year"].max(), 30)
y_range = np.linspace(combined_df["avg_comments_per_user"].min(), combined_df["avg_comments_per_user"].max(), 30)
x, y = np.meshgrid(x_range, y_range)
z = (-w[0] * x - w[1] * y - b) / w[2]  # Solve for z

# Density calculation for Dataset 1
xyz1 = subsampled_df[["year", "avg_comments_per_user", "distinct_videos_commented"]].values.T
kde1 = gaussian_kde(xyz1)(xyz1)
subsampled_df["density"] = kde1

# Density calculation for Dataset 2
xyz2 = subsampled_df_sus[["year", "avg_comments_per_user", "distinct_videos_commented"]].values.T
kde2 = gaussian_kde(xyz2)(xyz2)
subsampled_df_sus["density"] = kde2

# Create the 3D Scatter Plot
fig = go.Figure()

# Add Dataset 1 (Normal Users) with density-based coloring
fig.add_trace(go.Scatter3d(
    x=subsampled_df["year"],
    y=subsampled_df["avg_comments_per_user"],
    z=subsampled_df["distinct_videos_commented"],
    mode='markers',
    marker=dict(
        size=3,
        color=subsampled_df["density"],  # Color based on density
        colorscale="Viridis",  # Blue-Green color scale
        opacity=0.6
    ),
    name="Normal Users"
))

# Add Dataset 2 (Suspicious Users) with density-based coloring
fig.add_trace(go.Scatter3d(
    x=subsampled_df_sus["year"],
    y=subsampled_df_sus["avg_comments_per_user"],
    z=subsampled_df_sus["distinct_videos_commented"],
    mode='markers',
    marker=dict(
        size=3,
        color=subsampled_df_sus["density"],  # Color based on density
        colorscale="Jet",  # Yellow-to-Red color scale
        opacity=0.8
    ),
    name="Suspicious Users"
))

# Add the SVM hyperplane
fig.add_trace(go.Surface(
    x=x, y=y, z=z,
    colorscale=[[0, 'lightgrey'], [1, 'lightgrey']],  # Single light-grey color
    opacity=0.5,
    showscale=False,  # Remove colorbar for hyperplane
    name="SVM Hyperplane"
))

# Add a box with the hyperplane metrics
fig.add_annotation(
    text=f"<b>SVM Hyperplane Metrics</b><br>"
         f"Equation: {w[0]:.2f}*x1 + {w[1]:.2f}*x2 + {w[2]:.2f}*x3 + {b:.2f} = 0",
    showarrow=False,
    xref="paper", yref="paper",
    x=0.05, y=0.95,  # Position: top-left corner
    bordercolor="black", borderwidth=1,
    bgcolor="white", font=dict(size=12)
)

# Update layout
fig.update_layout(
    title="",
    scene=dict(
        xaxis=dict(title="Year", tickvals=list(range(2005, 2020)), autorange="reversed"),
        yaxis=dict(title="Average Comments/User", range=[0, 50]),
        zaxis=dict(title="Distinct Videos Commented", range=[0, 1000])
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the plot
fig.show()

# Save the plot as an HTML file
fig.write_html("./image_aurel/3D_hyperplane_density.html")


___

## <p style="text-align: center"> <span style="text-decoration: underline"> **Type-2 Bot Analysis** </span> </p>

#### - **Data Preprocessing and Loading**

In [ ]:
combine = False

if combine:

    datasets = ['normal', 'suspicious']

    for dataset in datasets:
        print(f"reading partial files '{dataset}_i.parquet'...")

        # List all Parquet files
        parquet_files = glob.glob(f'./data/data_type2/{dataset}_users_*.parquet')

        # Read and concatenate all Parquet files
        combined = pl.concat([pl.read_parquet(file) for file in parquet_files])
        combined.write_parquet(f'./data/data_type2/combi_{dataset}_dataset2.parquet')

In [ ]:
df_normal_2 = pl.read_parquet('./data/data_type2/combi_normal_dataset2.parquet')
df_sus_2 = pl.read_parquet('./data/data_type2/combi_suspicious_dataset2.parquet')

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **Do Bots Comment On More Videos ?** </span> </p>

In [ ]:
df_norm = pd.read_parquet(df_normal_2)[0] # Just need 1 df for plot
df_sus =  pd.read_parquet(df_sus_2)[0]
df_sus = df_sus[df_sus["videos_commented"] >= 10]

# Adding a column to discriminate between the 2 df
df_norm["category"]='normal'
df_sus["category"]='suspicious'

# Concatenating for the plot
df = pd.concat([df_norm[::5],df_sus[df_sus["videos_commented"] < 30]])

In [ ]:
sns.set_theme(style="darkgrid")
sns.violinplot(x="category", y="videos_commented", hue="category", data=df, palette="Pastel1", split=True)
plt.show()

In [ ]:
fig = go.Figure()

for category in ['normal', 'suspicious']:
    fig.add_trace(go.Violin(x=df['category'][df['category'] == category],
                            y=df['videos_commented'][df['category'] == category],
                            name=category,
                            box_visible=True,
                            meanline_visible=True))

fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay', yaxis=dict(
        title=dict(
            text="Number of videos commented"
        )
    ),)
fig.write_html("images/dist_num_comments_norm_sus.html")
#fig.show()

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How Does The Number of Comments and Videos Commented Vary Over Time ?** </span> </p>

In [ ]:
dataset = 'suspicious'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_files_type_2 = glob.glob(f'./data/data_type2/{dataset}_*.parquet')

In [ ]:
# Create and Save Time Series Data

result = pl.concat([pl.scan_parquet(dataset).group_by('upload_date').agg([
                    pl.col("videos_commented").sum(),
                    pl.col("author").unique().count()]).collect(streaming=True)
                    for dataset in tqdm(parquet_files_type_2, desc='Processing chunks')
                    ]).group_by('upload_date').agg([pl.col("videos_commented").sum(),
                    pl.col("author").unique().count()]).rename({'author' : 'distinct_bots'})

result = result.with_columns(comments_per_bot = result['videos_commented'] / result['distinct_bots'])
result.write_parquet('./data/data_type2/time_series_type_2.parquet')

In [ ]:
# Load Time Series Data

result = pl.read_parquet('./data/data_type2/time_series_type_2.parquet')
result = result.sort('upload_date')

In [ ]:
# Group by month
result_monthly = result.to_pandas().groupby(pd.Grouper(key='upload_date', freq='MS')).agg(
    videos_commented=('videos_commented', 'sum'),
    distinct_bots=('distinct_bots', 'sum')  # Use sum for distinct_bots
).reset_index()[:-1] #Getting rid of the last month as it is in complete

# Calculate comments per bot per month
result_monthly['comments_per_bot'] = result_monthly['videos_commented'] / result_monthly['distinct_bots']

# Plot the monthly data
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot videos commented per month
ax1.plot(result_monthly['upload_date'], result_monthly['videos_commented'], color='tab:blue', label='Videos Commented (Monthly)', marker='o')
ax1.set_xlabel('Month')
ax1.set_ylabel('Videos Commented', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create second y-axis to plot comments per bot per month
ax2 = ax1.twinx()
ax2.plot(result_monthly['upload_date'], result_monthly['comments_per_bot'], color='tab:red', label='Comments per Bot (Monthly)', marker='x')
ax2.set_ylabel('Comments per Bot', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Title and layout adjustments
plt.title('Bot Comments and Activity Trends (Grouped by Month)')
fig.tight_layout()  # Adjust layout to fit everything
plt.xticks(rotation=45)

# Show the plot
plt.show()

In [ ]:
# Group by year
result_yearly = result.to_pandas().groupby(pd.Grouper(key='upload_date', freq='YS')).agg(
    videos_commented=('videos_commented', 'sum'),
    distinct_bots=('distinct_bots', 'sum')  # Use sum for distinct_bots
).reset_index()[:-1] #Getting rid of the last year as it is in complete

# Calculate comments per bot per year
result_yearly['comments_per_bot'] = result_yearly['videos_commented'] / result_yearly['distinct_bots']

# Plot the yearly data
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot videos commented per year
ax1.plot(result_yearly['upload_date'], result_yearly['videos_commented'], color='tab:blue', label='Videos Commented (yearly)', marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('Videos Commented', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create second y-axis to plot comments per bot per year
ax2 = ax1.twinx()
ax2.plot(result_yearly['upload_date'], result_yearly['comments_per_bot'], color='tab:red', label='Comments per Bot (yearly)', marker='x')
ax2.set_ylabel('Comments per Bot', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Title and layout adjustments
plt.title('Bot Comments and Activity Trends (Grouped by Year)')
fig.tight_layout()  # Adjust layout to fit everything
plt.xticks(rotation=45)

# Show the plot
plt.show()

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **What Is a Typical Lifetime For a Bot ?** </span> </p>

#### - **Suspicious Users**

#### *1st Definition of Lifetime*

In [ ]:
lifetime_df = pl.concat([pl.scan_parquet(dataset).group_by("author").agg([
        pl.col("upload_date").min().alias("min"),
        pl.col("upload_date").max().alias("max")
        ]).collect(streaming=True)  
    for dataset in tqdm(parquet_files_type_2, desc='Processing df')])

In [ ]:
# To make sure that we have the min and max of all users
lifetime_df = lifetime_df.group_by("author").agg([
                            pl.col("min").min().alias("min"),
                            pl.col("max").max().alias("max")
                            ])

In [ ]:
# Computing the lifetime for each accounts and Saves the data

date_diff = (lifetime_df['max'] - lifetime_df['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_df = lifetime_df.with_columns([
    (date_diff + one_day).alias("lifetime")
])

lifetime_df.write_parquet('./data/data_type2/lifetime_sus.parquet')

In [ ]:
lifetime_df = pl.read_parquet('./data/data_type2/lifetime_sus.parquet')

# Extracting the lifetime and converting it to float64 for plotting
lifetime = lifetime_df.select(
    total_days = pl.col.lifetime.dt.total_days(),
)

#Plot the distribution of bot lifetimes
plt.figure(figsize=(10, 6))
plt.hist(lifetime, bins=100, color='blue', edgecolor='black')
plt.title("Distribution of Bots' Lifetime (in Days)")
plt.xlabel('Lifetime (Days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#### *2nd Definition of Lifetime*

In [ ]:
lifetime_sus_df = pl.concat([pl.scan_parquet(dataset).filter(pl.col('videos_commented')>=10).group_by("author")
        .agg([
        pl.col("upload_date").min().alias("min"),
        pl.col("upload_date").max().alias("max")
        ]).collect(streaming=True)  
    for dataset in tqdm(parquet_files_type_2, desc='Processing df')])

In [ ]:
# To make sure that we have the min and max of all users
lifetime_sus_df = lifetime_sus_df.group_by("author").agg([
                            pl.col("min").min().alias("min"),
                            pl.col("max").max().alias("max")
                            ])

In [ ]:
# Compute the lifetime for each account and save the data

date_diff = (lifetime_sus_df['max'] - lifetime_sus_df['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_sus_df = lifetime_sus_df.with_columns([
    (date_diff + one_day).alias("lifetime")
])

lifetime_sus_df.write_parquet('./data/data_type2/lifetime_sus_sus_days.parquet')

In [ ]:
lifetime_sus_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_days.parquet')

# Extracting the lifetime and converting it to float64 for plotting
lifetime = lifetime_sus_df.select(
    total_days = pl.col.lifetime.dt.total_days(),
)

#Plot the distribution of bot lifetimes
plt.figure(figsize=(10, 6))
plt.hist(lifetime, bins=100, color='blue', edgecolor='black')
plt.title("Distribution of Bots' suspicious Lifetime (in Days)")
plt.xlabel('Lifetime (Days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#### *3rd Definition of Lifetime*

In [ ]:
lifetime_sus_2_df = lifetime_sus_df.clone()

lifetime_sus_2_df = lifetime_sus_2_df.sort("author").with_columns(
    max = lifetime_df.sort("author")["max"])

In [ ]:
# Compute the lifetime for each account and save the data

date_diff = (lifetime_sus_2_df['max'] - lifetime_sus_2_df['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_sus_2_df = lifetime_sus_2_df.with_columns([
    (date_diff + one_day).alias("lifetime")
])

lifetime_sus_2_df.write_parquet('./data/data_type2/lifetime_sus_sus_2_days.parquet')

In [ ]:
lifetime_sus_2_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_2_days.parquet')

# Extracting the lifetime and converting it to float64 for plotting
lifetime = lifetime_sus_2_df.select(
    total_days = pl.col.lifetime.dt.total_days(),
)

#Plot the distribution of bot lifetimes
plt.figure(figsize=(10, 6))
plt.hist(lifetime, bins=100, color='blue', edgecolor='black')
plt.title("Distribution of Bots' suspicious Lifetime (in Days)")
plt.xlabel('Lifetime (Days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#### - **Normal Users**

#### *1st Definition of Lifetime*

In [ ]:
dataset = 'normal'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_normal_type_2 = glob.glob(f'./data/data_type2/{dataset}_*.parquet')

In [ ]:
lifetime_normal_df = pl.concat([pl.scan_parquet(dataset).group_by("author").agg([
        pl.col("upload_date").min().alias("min"),
        pl.col("upload_date").max().alias("max")
        ]).collect(streaming=True)  
    for dataset in tqdm(parquet_normal_type_2, desc='Processing df')])

In [ ]:
lifetime_normal_df = lifetime_normal_df.group_by("author").agg([
                            pl.col("min").min().alias("min"),
                            pl.col("max").max().alias("max")
                            ])

In [ ]:
# Compute the lifetime for each account and save the data

date_diff = (lifetime_normal_df['max'] - lifetime_normal_df['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_normal_df = lifetime_normal_df.with_columns([
    (date_diff + one_day).alias("lifetime")
])

lifetime_normal_df.write_parquet('./data/data_type2/lifetime_normal.parquet')

In [ ]:
lifetime_normal_df = pl.read_parquet('./data/data_type2/lifetime_normal.parquet')

# Extracting the lifetime and converting it to float64 for plotting
lifetime = lifetime_normal_df.select(
    total_days = pl.col.lifetime.dt.total_days(),
)

#Plot the distribution of bot lifetimes
plt.figure(figsize=(10, 6))
plt.hist(lifetime, bins=100, color='blue', edgecolor='black')
plt.title("Distribution of Normal users' Lifetime (in Days)")
plt.xlabel('Lifetime (Days)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#### - **Comparison Normal vs Bots**

#### *1st Definition of Lifetime*

In [ ]:
# Adding a column to discriminate between the 2 df
lifetime_df = lifetime_df.with_columns(pl.lit('suspicious').alias("category"))
lifetime_normal_df = lifetime_normal_df.with_columns(pl.lit('normal').alias("category"))

# Concatenating for the plot
df = pl.concat([lifetime_normal_df,lifetime_df]).to_pandas()

# Converting datetime to float
df["lifetime"] = df["lifetime"]/np.timedelta64(1,'D')

In [ ]:
# Grouped violinplot
sns.set_theme(style="darkgrid")

# Changing the order of the first two parts of the palette so normal is blue and sus is red
modified_palette = sns.color_palette("Pastel1")
modified_palette[0], modified_palette[1] = modified_palette[1], modified_palette[0]

sns.violinplot(x="category", y="lifetime", hue="category", data=df[::2], 
               palette=modified_palette, split=True)

plt.show()

#### *2nd Definition of Lifetime*

In [ ]:
# Adding a column to discriminate between the 2 df
lifetime_sus_df = lifetime_sus_df.with_columns(pl.lit('suspicious').alias("category"))

# Concatenating for the plot
df = pl.concat([lifetime_normal_df,lifetime_sus_df]).to_pandas()

# Converting datetime to float
df["lifetime"] = df["lifetime"]/np.timedelta64(1,'D')

In [ ]:
# Grouped violinplot
sns.set_theme(style="darkgrid")

# Changing the order of the first two parts of the palette so normal is blue and sus is red
modified_palette = sns.color_palette("Pastel1")
modified_palette[0], modified_palette[1] = modified_palette[1], modified_palette[0]

sns.violinplot(x="category", y="lifetime", hue="category", data=df[::2], 
               palette=modified_palette, split=True)
plt.show()

#### *3rd Definition of Lifetime*

In [ ]:
# Adding a column to discriminate between the 2 df
lifetime_sus_2_df = lifetime_sus_2_df.with_columns(pl.lit('suspicious').alias("category"))

# Concatenating for the plot
df = pl.concat([lifetime_normal_df,lifetime_sus_2_df]).to_pandas()

# Converting datetime to float
df["lifetime"] = df["lifetime"]/np.timedelta64(1,'D')

In [ ]:
# Grouped violinplot
sns.set_theme(style="darkgrid")

# Changing the order of the first two parts of the palette so normal is blue and sus is red
modified_palette = sns.color_palette("Pastel1")
modified_palette[0], modified_palette[1] = modified_palette[1], modified_palette[0]

sns.violinplot(x="category", y="lifetime", hue="category", data=df[::2], 
               palette=modified_palette, split=True)
plt.show()

#### - **Plotly Plots** (Very Heavy)

In [ ]:
lifetime_df = pl.read_parquet('./data/data_type2/lifetime_sus.parquet')
lifetime_normal_df = pl.read_parquet('./data/data_type2/lifetime_normal.parquet')
lifetime_sus_2_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_2_days.parquet')
lifetime_sus_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_days.parquet')

# Adding a column to discriminate between the df
lifetime_normal_df = lifetime_normal_df.with_columns(pl.lit('normal').alias("category"))
lifetime_df = lifetime_df.with_columns(pl.lit('suspicious1').alias("category"))
lifetime_sus_df = lifetime_sus_df.with_columns(pl.lit('suspicious2').alias("category"))
lifetime_sus_2_df = lifetime_sus_2_df.with_columns(pl.lit('suspicious3').alias("category"))

In [ ]:
df = pl.concat([lifetime_df[["lifetime",'category']].with_columns(type_life = 1),
           lifetime_normal_df[["lifetime",'category'
                               ]].with_columns(
                                   type_life = 1).replace_column(
                                                             1,pl.Series("category",['normal1']*len(lifetime_normal_df))),
           lifetime_sus_df[["lifetime",'category']].with_columns(type_life = 2),
           lifetime_normal_df[["lifetime",'category'
                               ]].with_columns(
                                   type_life = 2).replace_column(
                                                             1,pl.Series("category",['normal2']*len(lifetime_normal_df))),
           lifetime_sus_2_df[["lifetime",'category']].with_columns(type_life = 3),
           lifetime_normal_df[["lifetime",'category'
                               ]].with_columns(
                                   type_life = 3).replace_column(
                                                             1,pl.Series("category",['normal3']*len(lifetime_normal_df))),
           ]).to_pandas()[::20]

df["lifetime"] = df["lifetime"]/np.timedelta64(1,'D')

In [ ]:

# Create subplots
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)

# Add traces to the first subplot
fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'normal1'],
                        y=df['lifetime'][df['category'] == 'normal1'],
                        legendgroup='Yes', scalegroup='normal1', name='Normal',
                        side='negative',
                        line_color='blue',
                        width=0.8),  # Increase width
              row=1, col=1)

fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'suspicious1'],
                        y=df['lifetime'][df['category'] == 'suspicious1'],
                        legendgroup='No', scalegroup='normal1', name='Suspicious',
                        side='positive',
                        line_color='orange',
                        width=0.8),  # Increase width
              row=1, col=1)

# Add traces to the second subplot
fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'normal2'],
                        y=df['lifetime'][df['category'] == 'normal2'],
                        legendgroup='Yes', scalegroup='normal2', name='Normal',
                        side='negative',
                        line_color='blue',
                        width=0.8,
                        showlegend=False),  # Increase width
              row=1, col=2)

fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'suspicious2'],
                        y=df['lifetime'][df['category'] == 'suspicious2'],
                        legendgroup='No', scalegroup='normal2', name='Suspicious',
                        side='positive',
                        line_color='orange',
                        width=0.8,
                        showlegend=False),  # Increase width
              row=1, col=2)

# Add traces to the third subplot
fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'normal3'],
                        y=df['lifetime'][df['category'] == 'normal3'],
                        legendgroup='Yes', scalegroup='normal3', name='Normal',
                        side='negative',
                        line_color='blue',
                        width=0.8,
                        showlegend=False),  # Increase width
              row=1, col=3)

fig.add_trace(go.Violin(x=df['type_life'][df['category'] == 'suspicious3'],
                        y=df['lifetime'][df['category'] == 'suspicious3'],
                        legendgroup='No', scalegroup='normal3', name='Suspicious',
                        side='positive',
                        line_color='orange',
                        width=0.8,
                        showlegend=False),  # Increase width
              row=1, col=3)

# Update traces and layout
fig.update_traces(meanline_visible=True)
fig.update_layout(
    violingap=0.1,  # Adjust the gap between violins if needed
    violinmode='overlay',
    yaxis_title="Lifetime"
)

# Center the x-axis title for each subplot
for i in range(1, 4):
    fig.update_xaxes(title_text="Type of Lifetime", title_standoff=20, row=1, col=i)

fig.write_html("./images/violin_plots_lifetime.html")
#fig.show()


#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How Does Commenting Frequency Affects Lifetime ?** </span> </p>

In [ ]:
lifetime_df = pl.read_parquet('./data/data_type2/lifetime_sus.parquet')
lifetime_sus_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_days.parquet')
lifetime_sus_2_df = pl.read_parquet('./data/data_type2/lifetime_sus_sus_2_days.parquet')
numb_app_df = pl.read_parquet('./data/data_type2/numb_sus_days.parquet')

lifetime_sus_df = lifetime_sus_df.rename({'lifetime' : 'lifetime_sus'})
lifetime_sus_2_df = lifetime_sus_2_df.rename({'lifetime' : 'lifetime_sus_2'})

In [ ]:
freq_effect_df = lifetime_df[['author','lifetime']].join(other = lifetime_sus_df[['author','lifetime_sus']],
                                                    on = 'author').join(
                                                        other=lifetime_sus_2_df[['author','lifetime_sus_2']],
                                                          on ='author'
                                                    ).join(other=numb_app_df, on ='author')

freq_effect_df = freq_effect_df.rename({'count' : 'numb_day_sus'})

In [ ]:
# Converting the datetime index to float and filtering for the plot
lifetime = freq_effect_df.filter(freq_effect_df["numb_day_sus"] <=50).select(
    total_days = pl.col.lifetime.dt.total_days(),
)

lifetime_sus = freq_effect_df.filter(freq_effect_df["numb_day_sus"] <=50).select(
    total_days = pl.col.lifetime_sus.dt.total_days(),
)

lifetime_sus_2 = freq_effect_df.filter(freq_effect_df["numb_day_sus"] <=50).select(
    total_days = pl.col.lifetime_sus_2.dt.total_days(),
)

# Extracting the number of suspicious days
numb_day_sus = freq_effect_df.filter(freq_effect_df["numb_day_sus"] <=50)['numb_day_sus']

In [ ]:
# 2D density + marginal distribution:
sns.jointplot(x=numb_day_sus[::50], 
              y=lifetime['total_days'][::50],
              cmap="Blues", shade=True, kind='kde')
plt.show()

In [ ]:
P = freq_effect_df.select(pl.corr('lifetime','numb_day_sus', method='pearson')).item()
S = freq_effect_df.select(pl.corr('lifetime','numb_day_sus', method='spearman')).item()

print(f"Pearson's coeff : {P}")
print(f"Spearman's coeff : {S}")

In [ ]:
# 2D density + marginal distribution:
sns.jointplot(x=numb_day_sus[::50], 
              y=lifetime_sus['total_days'][::50],
              cmap="Blues", shade=True, kind='kde')
plt.show()

In [ ]:
P = freq_effect_df.select(pl.corr('lifetime_sus','numb_day_sus', method='pearson')).item()
S = freq_effect_df.select(pl.corr('lifetime_sus','numb_day_sus', method='spearman')).item()

print(f"Pearson's coeff : {P}")
print(f"Spearman's coeff : {S}")

In [ ]:
# 2D density + marginal distribution:
sns.jointplot(x=numb_day_sus[::50], 
              y=lifetime_sus_2['total_days'][::50],
              cmap="Blues", shade=True, kind='kde')
plt.show()

In [ ]:
P = freq_effect_df.select(pl.corr('lifetime_sus_2','numb_day_sus', method='pearson')).item()
S = freq_effect_df.select(pl.corr('lifetime_sus_2','numb_day_sus', method='spearman')).item()

print(f"Pearson's coeff : {P}")
print(f"Spearman's coeff : {S}")

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **How Do Normal Users' and Bots' Metrics Correlate ?** </span> </p>

#### - **Suspicious Users**

In [ ]:
dataset = 'suspicious'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_files_type_1 = glob.glob(f'./data/data_type1/{dataset}_*.parquet')

In [ ]:
len_type_1_sus = pl.concat([pl.scan_parquet(file).select('author').unique().select(pl.len()).collect() for file in tqdm(parquet_files_type_1)]).sum().item()

In [ ]:
df_sus_type_1=pl.concat([pl.scan_parquet(file) 
                            for file in parquet_files_type_1])

df_sus_type_1 = df_sus_type_1.with_columns([
    pl.col("upload_date").cast(pl.Date),  # Ensure it's in datetime format
    pl.col("upload_date").dt.year().alias("year")  # Extract year
])

In [ ]:
len_type_1_sus = 7_586_337

# Define chunk size
chunk_size_sus = 500_000  # Adjust based on memory constraints

# Initialize lists to collect results
unique_users_results_sus = []
chunk_results_sus = []

# Iterate through the dataset in chunks
for start in tqdm(range(0, len_type_1_sus, chunk_size), desc='Processing 1'):
    # Slice the DataFrame for the current chunk
    chunk_sus = df_sus_type_1.slice(start, chunk_size).collect(streaming=True)

    # Fill missing values in specific columns (only if they exist)
    columns_to_fill_sus = ["comments", "likes", "replies", "view_count"]
    for col_sus in tqdm(columns_to_fill_sus, desc='Filling column'):
        if col_sus in chunk_sus.columns:
            chunk_sus = chunk_sus.with_columns(pl.col(col_sus).fill_null(0))

    # Aggregate metrics for this chunk
    chunk_metrics_sus = (
        chunk_sus.group_by("year")
        .agg([
            pl.col("comments").sum().alias("total_comments"),
            pl.col("likes").sum().alias("total_likes"),
            pl.col("replies").sum().alias("total_replies"),
            #pl.col("view_count").sum().alias("total_views"),
            #pl.col("video_id").count().alias("total_videos"),
        ])
    )
    chunk_results_sus.append(chunk_metrics_sus)

# Combine all chunk results for aggregated metrics
final_metrics_sus = pl.concat(chunk_results_sus).group_by("year").sum()

# Iterate through the dataset again for unique users
for start in tqdm(range(0, len_type_1_sus, chunk_size), desc='Processing 2'):
    # Slice the DataFrame for the current chunk
    chunk_sus = df_sus_type_1.slice(start, chunk_size).collect(streaming=True)

    # Calculate unique users per year
    unique_users_sus = (
        chunk_sus.group_by("year")
        .agg(pl.col("author").n_unique().alias("unique_users"))
    )
    unique_users_results_sus.append(unique_users_sus)

# Combine all unique users results
unique_users_combined_sus = pl.concat(unique_users_results_sus).group_by("year").sum()

# Merge the aggregated metrics with unique users
final_result_sus = final_metrics_sus.join(unique_users_combined_sus, on="year", how="left")

# Compute comments per user
final_result_sus = final_result_sus.with_columns(
    (pl.col("total_comments") / pl.col("unique_users")).alias("comments_per_user")
)
# Sort the DataFrame by year
final_result_sus = final_result_sus.sort("year")

#### - **Normal Users**

In [ ]:
dataset = 'normal'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_files_type_1 = glob.glob(f'./data/data_type1/{dataset}_*.parquet')

In [ ]:
len_type_1_normal = pl.concat([pl.scan_parquet(file).select('author').unique().select(pl.len()).collect() for file in tqdm(parquet_files_type_1)]).sum().item()

In [ ]:
df_normal_type_1=pl.concat([pl.scan_parquet(file) 
                            for file in parquet_files_type_1])

df_normal_type_1 = df_normal_type_1.with_columns([
    pl.col("upload_date").cast(pl.Date),  # Ensure it's in datetime format
    pl.col("upload_date").dt.year().alias("year")  # Extract year
])

In [ ]:
len_type_1_normal = 5_287_367

# Define chunk size
chunk_size = 1_000_000  # Adjust based on memory constraints

# Initialize lists to collect results
unique_users_results = []
chunk_results = []

# Iterate through the dataset in chunks
for start in tqdm(range(0, len_type_1_normal, chunk_size), desc='Processing 1'):
    # Slice the DataFrame for the current chunk
    chunk = df_normal_type_1.slice(start, chunk_size).collect(streaming=True)

    # Fill missing values in specific columns (only if they exist)
    columns_to_fill = ["comments", "likes", "replies", "view_count"]
    for col in tqdm(columns_to_fill, desc='Filling column'):
        if col in chunk.columns:
            chunk = chunk.with_columns(pl.col(col).fill_null(0))

    # Aggregate metrics for this chunk
    chunk_metrics = (
        chunk.group_by("year")
        .agg([
            pl.col("comments").sum().alias("total_comments"),
            pl.col("likes").sum().alias("total_likes"),
            pl.col("replies").sum().alias("total_replies"),
            #pl.col("view_count").sum().alias("total_views"),
            #pl.col("video_id").count().alias("total_videos"),
        ])
    )
    chunk_results.append(chunk_metrics)

# Combine all chunk results for aggregated metrics
final_metrics = pl.concat(chunk_results).group_by("year").sum()

# Iterate through the dataset again for unique users
for start in tqdm(range(0, len_type_1_normal, chunk_size), desc='Processing 2'):
    # Slice the DataFrame for the current chunk
    chunk = df_normal_type_1.slice(start, chunk_size).collect(streaming=True)

    # Calculate unique users per year
    unique_users = (
        chunk.group_by("year")
        .agg(pl.col("author").n_unique().alias("unique_users"))
    )
    unique_users_results.append(unique_users)

# Combine all unique users results
unique_users_combined = pl.concat(unique_users_results).group_by("year").sum()

# Merge the aggregated metrics with unique users
final_result = final_metrics.join(unique_users_combined, on="year", how="left")

# Compute comments per user
final_result = final_result.with_columns(
    (pl.col("total_comments") / pl.col("unique_users")).alias("comments_per_user")
)

#### - **Correlation Matrix**

In [ ]:
# Convert Polars DataFrames to Pandas DataFrames
df1_pd = final_result.to_pandas()
df2_pd = final_result_sus.to_pandas()

# Concatenate the two DataFrames
concatenated_df = pd.concat([df1_pd, df2_pd], axis=1, keys=["normal", "sus"])

# Compute the correlation matrix
correlation_matrix = concatenated_df.corr().abs()

# Get the number of columns
num_cols = len(correlation_matrix.columns)

# Slice the correlation matrix to get the top right quarter
# This will select the upper right quarter of the matrix
top_right_quarter = correlation_matrix.iloc[:num_cols//2, num_cols//2:]

# Plot the sliced correlation matrix using Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(top_right_quarter, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Top Right Quarter Correlation Map")
plt.show()

In [ ]:
# Convert the 'year' column to a date type (assuming the 'year' contains year only, adjust if needed)
df = final_result.with_columns(pl.col("year").cast(pl.Utf8).str.strptime(pl.Date, "%Y"))

# Extract the 'year' column as a pandas DataFrame
date_column = df.select("year").to_pandas()

# Drop the 'year' column from the DataFrame for scaling
df_without_date = df.drop("year").to_pandas()

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
scaled_array = scaler.fit_transform(df_without_date)

# Convert the scaled array back to a Pandas DataFrame
scaled_df_pd = pd.DataFrame(scaled_array, columns=df_without_date.columns)

# Add the 'year' column back to the scaled DataFrame
scaled_df_pd["year"] = date_column

# Convert the Pandas DataFrame back to a Polars DataFrame
scaled_df = pl.DataFrame(scaled_df_pd)

final_result = scaled_df.sort('year')

In [ ]:
# Convert the 'year' column to a date type (assuming the 'year' contains year only, adjust if needed)
df = final_result_sus.with_columns(pl.col("year").cast(pl.Utf8).str.strptime(pl.Date, "%Y"))

# Extract the 'year' column as a pandas DataFrame
date_column = df.select("year").to_pandas()

# Drop the 'year' column from the DataFrame for scaling
df_without_date = df.drop("year").to_pandas()

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
scaled_array = scaler.fit_transform(df_without_date)

# Convert the scaled array back to a Pandas DataFrame
scaled_df_pd = pd.DataFrame(scaled_array, columns=df_without_date.columns)

# Add the 'year' column back to the scaled DataFrame
scaled_df_pd["year"] = date_column

# Convert the Pandas DataFrame back to a Polars DataFrame
scaled_df = pl.DataFrame(scaled_df_pd)

final_result_sus = scaled_df.sort('year')

In [ ]:
# Convert Polars DataFrames to Pandas DataFrames
df1_pd = final_result.to_pandas()
df2_pd = final_result_sus.to_pandas()

# Concatenate the two DataFrames
concatenated_df = pd.concat([df1_pd, df2_pd], axis=1, keys=["normal", "sus"])

# Reset the index to flatten the hierarchical index
concatenated_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in concatenated_df.columns]

# Compute the correlation matrix
correlation_matrix = concatenated_df.corr().abs()

# Get the number of columns
num_cols = len(correlation_matrix.columns)

# Slice the correlation matrix to get the top right quarter
# This will select the upper right quarter of the matrix
top_right_quarter = correlation_matrix.iloc[:num_cols//2-1, num_cols//2:-1]

# Plot the sliced correlation matrix using Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(top_right_quarter, annot=True, cmap='coolwarm', fmt='.2f', vmin=0, vmax=1)
plt.title("Correlation map of the evolution of normal and suspicious users")
plt.savefig('./images/corr_map_evol_normal_sus_T1.svg', bbox_inches='tight')
plt.show()

In [ ]:
pio.templates.default = "plotly_white"

fig = go.Figure(data=go.Heatmap(
    z=top_right_quarter.values,
    x=top_right_quarter.columns,
    y=top_right_quarter.index,
    colorscale=px.colors.diverging.RdBu[::-1],
    zmin=0,
    zmax=1,
    text = top_right_quarter.round(2).values,
    texttemplate="%{text}", hovertemplate=None
))

fig.update_layout(
    xaxis_nticks=36,
    xaxis_title='Suspicious',
    yaxis_title='Normal' 
)
fig.write_html("./images/heatmap_norm_sus.html")
fig.show()

___

## <p style="text-align: center"> <span style="text-decoration: underline"> **Intersection of Type-1 and 2 Bot Analysis** </span> </p>

#### - **Data Processing**

In [ ]:
dataset = 'suspicious'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_files_type_1 = glob.glob(f'./data/data_type1/{dataset}_*.parquet')
parquet_files_type_2 = glob.glob(f'./data/data_type2/{dataset}_*.parquet')

# Read and concatenate all author columns of Parquet files
authors_type_1 = pl.concat([pl.scan_parquet(file).select("author").collect() 
                            for file in parquet_files_type_1]).unique()

authors_type_2 = pl.concat([pl.scan_parquet(file).select('author').unique()
                            for file in parquet_files_type_2])

In [ ]:
len_type_2 = pl.concat([pl.scan_parquet(file).select('author').unique().select(pl.len()).collect() for file in parquet_files_type_2]).sum()

In [ ]:
chunk_size = 1_000_000

# Setting the DF for both type of suspicious accounts
authors_1_2 = pl.DataFrame()

for start in tqdm(range(0, len_type_2, chunk_size), desc="Processing chunks"):

    chunk = authors_type_2.slice(start, chunk_size).collect()

	# Only keeping the unique ID to save RAM
    joined_chunk = authors_type_1.join(chunk, on="author", how="inner").unique()

    authors_1_2 = pl.concat([authors_1_2, joined_chunk])

authors_1_2 = authors_1_2.unique()

# Save the data
authors_1_2.write_parquet("./data/intersection/suspicious_users_1_2.parquet")

In [ ]:
# Load the data
authors_1_2 = pl.read_parquet("./data/intersection/suspicious_users_1_2.parquet")

In [ ]:
len_type_2 = 3845594
len_type_1 = len(authors_type_1.unique())
len_1_2 = len(authors_1_2)

print(f"Ratio of users in type 1 : {len_1_2/len_type_1*100}")
print(f"Ratio of users in type 2 : {len_1_2/len_type_2*100}")

In [ ]:
# To read partial Parquet files & create a single df

#dataset = 'normal_users'
dataset = 'suspicious'

print(f"reading partial files '{dataset}_i.parquet'...")

# List all Parquet files
parquet_files_type_1 = glob.glob(f'./data/data_type1/{dataset}_*.parquet')
parquet_files_type_2 = glob.glob(f'./data/data_type2/{dataset}_*.parquet')

# Read and concatenate all author columns of Parquet files
authors_type_1 = pl.concat([pl.scan_parquet(file).select(["author","upload_date"]).collect() 
                            for file in parquet_files_type_1])

authors_type_2 = pl.concat([pl.scan_parquet(file).select(['author','upload_date'])
                            for file in parquet_files_type_2])

In [ ]:
authors_1_2_date = authors_type_1.join(other=authors_1_2, on="author", how="inner")
authors_1_excl = authors_type_1.join(other=authors_1_2, on="author", how="anti")

In [ ]:
# Extracting the unique type 2 accounts
chunk_size = 1_000_000

# Setting the DF for both type of suspicious accounts
authors_2_excl = pl.DataFrame()

for start in tqdm(range(0, len_type_2, chunk_size), desc="Processing chunks"):

    chunk = authors_type_2.slice(start, chunk_size).collect()

	# Only keeping the authors that are only type 2
    anti_joined_chunk = chunk.join(other=authors_1_2, on="author", how="anti")

    authors_2_excl = pl.concat([authors_2_excl, anti_joined_chunk])

In [ ]:
authors_1_excl = authors_1_excl.with_columns([
    pl.col("upload_date").dt.year().alias("year"),
    pl.col("upload_date").dt.month().alias("month")
])

authors_2_excl = authors_2_excl.with_columns([
    pl.col("upload_date").dt.year().alias("year"),
    pl.col("upload_date").dt.month().alias("month")
])

authors_1_2_date = authors_1_2_date.with_columns([
    pl.col("upload_date").dt.year().alias("year"),
    pl.col("upload_date").dt.month().alias("month")
])

In [ ]:
# Group by year and month and sum the values
authors_1_excl = authors_1_excl.group_by(["year", "month"]).agg([
    pl.count()
])

authors_2_excl = authors_2_excl.group_by(["year", "month"]).agg([
    pl.count()
])

authors_1_2_date = authors_1_2_date.group_by(["year", "month"]).agg([
    pl.count()
])

In [ ]:
# Convert to a pandas df each type before concatenating them
# Replacing the columns year month by a unique datetime column
authors_1_excl = authors_1_excl.with_columns([
    (pl.col("year") * 100 + pl.col("month")).cast(pl.Utf8).alias("year_month")
]).sort("year_month")

authors_2_excl = authors_2_excl.with_columns([
    (pl.col("year") * 100 + pl.col("month")).cast(pl.Utf8).alias("year_month")
]).sort("year_month")

authors_1_2_date = authors_1_2_date.with_columns([
    (pl.col("year") * 100 + pl.col("month")).cast(pl.Utf8).alias("year_month")
]).sort("year_month")

In [ ]:
# dropping the useless columns
authors_1_excl = authors_1_excl.drop(["year",'month'])
authors_2_excl = authors_2_excl.drop(["year",'month'])
authors_1_2_date = authors_1_2_date.drop(["year",'month'])

In [ ]:
# Converting pandas
authors_1_excl = authors_1_excl.to_pandas()
authors_2_excl = authors_2_excl.to_pandas()
authors_1_2_date = authors_1_2_date.to_pandas()

In [ ]:
# Convert 'year_month' to datetime format
authors_1_excl['year_month'] = authors_1_excl['year_month'].apply(lambda x: datetime.strptime(x, "%Y%m"))
authors_2_excl['year_month'] = authors_2_excl['year_month'].apply(lambda x: datetime.strptime(x, "%Y%m"))
authors_1_2_date['year_month'] = authors_1_2_date['year_month'].apply(lambda x: datetime.strptime(x, "%Y%m"))

In [ ]:
authors_1_excl = authors_1_excl.set_index('year_month')
authors_2_excl = authors_2_excl.set_index('year_month')
authors_1_2_date = authors_1_2_date.set_index('year_month')

In [ ]:
authors_1_excl = authors_1_excl.rename(columns={"count":'authors_1_excl'})
authors_2_excl = authors_2_excl.rename(columns={"count":'authors_2_excl'})
authors_1_2_date = authors_1_2_date.rename(columns={"count":'authors_1_2'})


In [ ]:
# Creating a new df for the stacked area plot | Deleting last month because not complete
authors_evol_df = pd.concat([authors_1_excl,authors_2_excl,authors_1_2_date], axis=1).fillna(0)[:-1]

#### - **Plots**

In [ ]:
# Plot the stacked area chart with smoothing and custom colors
plt.figure(figsize=(6, 6))  # Set the figure size
plt.stackplot(authors_evol_df.index,
              authors_evol_df.values.T,
              labels=authors_evol_df.columns)
plt.xlabel('Year') # Add a label for the x-axis
plt.ylabel('Total number of suspicious accounts') # Add a label for the y-axis
plt.title('Stacked Area Chart of the different type of bots') # Add a title
plt.legend(loc='upper left') # Add a legend in the upper left corner of the plot

# Show the plot
plt.show()

In [ ]:
# Define custom colors for the countries
custom_colors = ["#003f5c","#2f4b7c","#665191","#a05195","#d45087","#f95d6a","#ff7c43","#ffa600"]

custom_colors = ["#d45087","#f95d6a","#ffa600"]

# Define the desired order of countries
desired_order = ["authors_1_2", "authors_1_excl", "authors_2_excl"]

# Reorder the columns of the pivot_df and custom_colors list
authors_evol_df = authors_evol_df[desired_order]


# Convert the datetime index to numeric values (nanoseconds since the Unix epoch)
x_numeric = authors_evol_df.index.astype(np.int64)

# Smooth the lines using spline interpolation
x_smooth_numeric = np.linspace(x_numeric.min(), x_numeric.max(), 300)
pivot_smooth = pd.DataFrame({author_type: make_interp_spline(x_numeric, authors_evol_df[author_type])(x_smooth_numeric)
                             for author_type in authors_evol_df.columns})

# Convert the smoothed numeric values back to datetime
x_smooth = pd.to_datetime(x_smooth_numeric)

# Plot the stacked area chart with smoothing and custom colors
plt.figure(figsize=(10, 6))  # Set the figure size
plt.stackplot(x_smooth,
              pivot_smooth.values.T,
              labels=pivot_smooth.columns,
              colors=custom_colors)
plt.xlabel('Year') # Add a label for the x-axis
plt.ylabel('Total number of suspicious accounts') # Add a label for the y-axis
#plt.yscale("symlog")
plt.title('Stacked Area Chart of the different type of bots') # Add a title
plt.legend(loc='upper left') # Add a legend in the upper left corner of the plot


# Show the plot
plt.show()

In [ ]:
#### ---------
#### Smoothing the graph
# Define custom colors for the countries
custom_colors = ["#d45087", "#f95d6a", "#ffa600"]

# Define the desired order of countries
desired_order = ["authors_1_2", "authors_1_excl", "authors_2_excl"]

# Reorder the columns of the pivot_df and custom_colors list
authors_evol_df = authors_evol_df[desired_order]

# Convert the datetime index to numeric values (nanoseconds since the Unix epoch)
x_numeric = authors_evol_df.index.astype(np.int64)

# Smooth the lines using spline interpolation
x_smooth_numeric = np.linspace(x_numeric.min(), x_numeric.max(), 300)
pivot_smooth = pd.DataFrame({author_type: make_interp_spline(x_numeric, authors_evol_df[author_type])(x_smooth_numeric)
                             for author_type in authors_evol_df.columns})

# Convert the smoothed numeric values back to datetime
x_smooth = pd.to_datetime(x_smooth_numeric)

### -------
### Getting the limits
start_time =authors_evol_df.index[0] - pd.Timedelta(weeks=4)
end_time = authors_evol_df.index[-1] +  pd.Timedelta(weeks=4)

### -------
### Making the plot

# Plot the stacked area chart with smoothing and custom colors
plt.figure(figsize=(12, 8))  # Set the figure size
plt.stackplot(x_smooth, pivot_smooth.values.T, labels=pivot_smooth.columns, colors=custom_colors)

# Allowing latex for equations
plt.rcParams['text.usetex'] = True

# `plt.gca()` function is used to obtain a reference to the current axes on which you plot your data
ax = plt.gca()

# Annotations for the values per year
def add_annotations_year(date):
    """
    Input: a year
    Apply: add to the graph the total wealth of all countries at a given date
           and a line from the bottom of the graph to the total value of wealth
    """
    
    # Calculate total amount of wealth at a given year
    y_end = int(authors_evol_df.loc[date].sum())
    
    # Set values in areas where the graph does not appear
    # Special case for 2021: we put it on the left instead of upper the line
    if date==authors_evol_df.index[-1]:
        date_text = date + pd.Timedelta(weeks=6)
        modif_yaxis = 20000
    else:
        date_text = date - pd.Timedelta(weeks=22)
        modif_yaxis = 20000
    
    # Add the values, with a specific position, in bold, black and a fontsize of 10
    plt.text(date_text,
             y_end+modif_yaxis,
             f'{y_end}',
             fontsize=15,
             color='black',
             fontweight = 'bold')
    
    # Add line 
    ax.plot([date, date], # x-axis position
            [0, y_end*1.05], # y-axis position (*1.05 is used to make a it little bit longer)
            color='black', # Color
            linewidth=1.3) # Width of the line
    
    # Add a point at the top of the line
    ax.plot(date, # x-axis position
            y_end*1.05, # y-axis position (*1.05 is used to make a it little bit longer)
            marker='o', # Style of the point
            markersize=5, # Size of the point
            color='black') # Color

# Add the line and the values for each of the following years
for date in ['2008-01-01', '2011-05-01','2013-08-01','2017-01-01','2019-09-01']:
    add_annotations_year(pd.Timestamp(date))
    

# Annotations for the values per country
def add_annotations_country(country, value_placement, amount, color):
    """
    Adds an annotation to a plot at a specific location with information about a country's amount in millions.

    Parameters:
        country (str): The name of the country for which the annotation is being added.
        value_placement (float): The vertical position where the annotation will be placed on the plot.
        amount (float): The amount in millions that will be displayed in the annotation.
        color (str): The color of the annotation text.
    """
    plt.text(pd.Timestamp('2020-01-01'), value_placement, f'{country} : {amount}', fontsize=15, color=color, fontweight='bold')

# We manually define the labels, values and position that will be displayed on the right of the graph
countries = [r'$\overline{T}_2$', r'$\overline{T}_1$', r"$T_{1,2}$"]
values_placement = [430000, 400000, 295000]
amounts = [int(authors_evol_df["authors_2_excl"][-1]), int(authors_evol_df["authors_1_excl"][-1]),
            int(authors_evol_df["authors_1_2"][-1])]
custom_colors.reverse()  # Makes sure the colors match the country concerned

# Iterate over all countries and add the name with the right value and color
for country, value, amount, color in zip(countries, values_placement, amounts, custom_colors):
    add_annotations_country(country, value, amount, color)


# Remove the y-axis frame (left, right and top spines)
#ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Remove the ticks and labels on the y-axis
#ax.tick_params(left=False, labelleft=False)
# Grid on the y-axis 
plt.grid(axis="y", zorder=-2, linestyle='--', color='gray', alpha=0.5) 

# Set the x-axis limits to ensure the text and plot fit well
plt.xlim(start_time, end_time)

# Set the y-axis limits to ensure the text and plot fit well
plt.ylim(0, 500000)

# Save the graph
plt.savefig("./images/different_types_of_bots.svg", bbox_inches='tight')

# Display the chart
plt.show()

___

#### <p style="text-align: center; margin-top: 0.5cm"> <span style="text-decoration: underline"> **BOT RANKING** </span> </p>


#### - Type-1 Bot Ranking

In [ ]:
# Compute lifetime of suspicious users

lifetime_df = df_sus.group_by("author").agg([
                pl.col("upload_date").min().alias("min"),
                pl.col("upload_date").max().alias("max")])

# Compute the difference
date_diff = (lifetime_df['max'] - lifetime_df['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_df = lifetime_df.with_columns([
    (date_diff + one_day).alias("lifetime")
])

In [ ]:
longuest_survival_type1=lifetime_df["lifetime"].max()
longest_survival_row_type1 = lifetime_df.filter(pl.col("lifetime") == pl.col("lifetime").max())

In [ ]:
# Total comments for each unique author across every day
total_comments_df = df_sus.group_by("author").agg([
    pl.col("comments").sum().alias("total_comments"),
])

# Max number of comments done in a day for each author
max_daily_comments_df = df_sus.group_by("author").agg([
    pl.col("comments").max().alias("max_daily_comments"),
])

# Combine the two metrics into a single DataFrame
author_metrics_df = total_comments_df.join(max_daily_comments_df, on="author")

# Extract the row with the maximum total comments
row_with_max_total_comments = author_metrics_df.filter(pl.col("total_comments") == pl.col("total_comments").max())

# Extract the row with the maximum daily comments
row_with_max_daily_comments = df_sus.filter(pl.col("comments") == pl.col("comments").max())

In [ ]:
# 1. Max number of different videos commented by an author in a day
max_videos_per_day = (
    df_sus.group_by(["author", "upload_date"])
    .agg([
        pl.col("video_id").n_unique().alias("unique_videos_commented")
    ])
    .group_by("author")
    .agg([
        pl.col("unique_videos_commented").max().alias("max_videos_in_a_day")
    ])
)

# 2. Max number of different videos commented across the entire dataset for each author
max_videos_overall = (
    df_sus.group_by("author")
    .agg([
        pl.col("video_id").n_unique().alias("max_videos_overall")
    ])
)

# Retrieve the row with the max number of different videos commented in a day
row_with_max_videos_in_a_day = max_videos_per_day.filter(
    pl.col("max_videos_in_a_day") == pl.col("max_videos_in_a_day").max()
)

# Retrieve the row with the max number of different videos commented across the entire dataset
row_with_max_videos_overall = max_videos_overall.filter(
    pl.col("max_videos_overall") == pl.col("max_videos_overall").max()
)
 
max_videos_per_day.filter(pl.col("max_videos_in_a_day")== pl.col("max_videos_in_a_day").max())

In [ ]:
# 1. Filter 1: Count days with more than 10 comments per day
filter_1_days = (
    df_sus.filter(pl.col("comments") > 10)
    .group_by("author")
    .agg(pl.col("upload_date").n_unique().alias("suspicious_days_filter_1"))
)

# 2. Filter 2: Count days with more than 10 unique videos commented
filter_2_days = (
    df_sus.group_by(["author", "upload_date"])
    .agg(pl.col("video_id").n_unique().alias("unique_videos_commented"))
    .filter(pl.col("unique_videos_commented") > 10)
    .group_by("author")
    .agg(pl.col("upload_date").n_unique().alias("suspicious_days_filter_2"))
)

# # 3. Combine results into one DataFrame
# suspicious_lifetime = filter_1_days.join(filter_2_days, on="author", how="outer").fill_null(0)

# 4. Extract rows with maximum suspicious days for each filter
max_suspicious_days_filter_1 = filter_1_days.filter(
    pl.col("suspicious_days_filter_1") == pl.col("suspicious_days_filter_1").max()
)
max_suspicious_days_filter_2 = filter_2_days.filter(
    pl.col("suspicious_days_filter_2") == pl.col("suspicious_days_filter_2").max()
)


#### - Type-2 Bot Ranking

In [ ]:
df_sus_type_2=pl.read_parquet('data/data_type2/combi_suspicious_dataset2.parquet')

lifetime_df_type2 = df_sus_type_2.group_by("author").agg([
                pl.col("upload_date").filter(pl.col('videos_commented')>=10).min().alias("min"),
                pl.col("upload_date").max().alias("max")])

In [ ]:
# Compute Lifetime

date_diff_type2 = (lifetime_df_type2['max'] - lifetime_df_type2['min']).alias("date_diff")

# Create the constant of one day to add
one_day = pl.duration(days=1).alias("one_day")

lifetime_df_type2 = lifetime_df_type2.with_columns([
    (date_diff_type2 + one_day).alias("lifetime")
])

In [ ]:
longuest_survival_type2=lifetime_df_type2["lifetime"].max()
longest_survival_row_type2 = lifetime_df_type2.filter(pl.col("lifetime") == pl.col("lifetime").max())

In [ ]:
inter_1_2=pl.read_parquet('data/suspicious_users_1_2.parquet')
recup_type_2= inter_1_2.join(df_sus, on="author", how="inner")

In [ ]:
# 1. Filter 1: Count days with more than 10 comments per day
max_suspicious_days_filter_1_type2 = (
    recup_type_2.filter(pl.col("comments") > 10)
    .group_by("author")
    .agg(pl.col("upload_date").n_unique().alias("suspicious_days_filter_1"))
    .filter(pl.col("suspicious_days_filter_1") == pl.col("suspicious_days_filter_1").max())
)

In [ ]:
filter_type2_on_type2=recup_type_2.filter(pl.col("comments") > 10).group_by("author").agg(pl.col("upload_date").n_unique().alias("suspicious_days_filter_1"))

In [ ]:
numb_app_df = pl.concat([pl.scan_parquet(dataset).filter(pl.col('videos_commented') >=10).group_by('author').count().collect(streaming=True)  
    for dataset in tqdm(parquet_files_type_2, desc='Processing df')
    ]).group_by('author').agg(pl.col("count").sum())

In [ ]:
# Find the row(s) with the maximum "count" meaning number of suspicious days in the type2 dataset
max_count_value = numb_app_df["count"].max()

# Retrieve the row(s) with the maximum count
rows_with_max_count = numb_app_df.filter(pl.col("count") == max_count_value)


In [ ]:
# Total comments for each unique author across every day
total_comments_df_type_2 = recup_type_2.group_by("author").agg([
    pl.col("comments").sum().alias("total_comments"),
])

# Max number of comments done in a day for each author
max_daily_comments_df_type2 = recup_type_2.group_by("author").agg([
    pl.col("comments").max().alias("max_daily_comments"),
])

# Combine the two metrics into a single DataFrame
author_metrics_df_type_2 = total_comments_df_type_2.join(max_daily_comments_df_type2, on="author")

# Extract the row with the maximum total comments
row_with_max_total_comments_type_2 = author_metrics_df_type_2.filter(pl.col("total_comments") == pl.col("total_comments").max())

# Extract the row with the maximum daily comments
row_with_max_daily_comments_type_2 = recup_type_2.filter(pl.col("comments") == pl.col("comments").max())


In [ ]:
# 2. Max number of different videos commented across the entire dataset for each author
max_videos_overall_type_2 = df_sus_type_2.group_by("author").agg(pl.col("videos_commented").sum().alias("max_videos_overall"))

# Retrieve the row with the max number of different videos commented in a day
row_with_max_videos_in_a_day_type_2 = df_sus_type_2["videos_commented"].max()

# Retrieve the row with the max number of different videos commented across the entire dataset
row_with_max_videos_overall = max_videos_overall_type_2["max_videos_overall"].max()


In [ ]:
# 2. Retrieve the row with the max number of videos commented in a day
row_with_max_videos_in_a_day_type_2 = df_sus_type_2.filter(
    pl.col("videos_commented") == pl.col("videos_commented").max()
)

# 3. Retrieve the row with the max number of videos commented across the entire dataset
row_with_max_videos_overall = max_videos_overall_type_2.filter(
    pl.col("max_videos_overall") == pl.col("max_videos_overall").max()
)

In [ ]:
#Retrieving all the 7 metrics for each top value ! 
            #champ1     #champ2     #champ3  #champ4
champions= [551221769, 260739815, 157898414, 303364156]

#Retrieve the lifetime: 
champ1_lifetime= lifetime_df.filter(pl.col("author") == 551221769)
print(champ1_lifetime)

champ1_lifetime_t2= lifetime_df_type2.filter(pl.col("author") == 551221769)
print(champ1_lifetime_t2)
#retrieve max comm in a day: 
row_with_max_daily_comments_champ1 = df_sus.filter(pl.col("author") == 551221769).filter(pl.col("comments") == pl.col("comments").max())
print(row_with_max_daily_comments_champ1)

#retrieve total number of comm: 
total_com_champ1=total_comments_df.filter(pl.col("author") == 551221769)
print(total_com_champ1)

#retrieve max # of different videos in a day
max_videos_per_day_champ1=max_videos_per_day.filter(pl.col("author") == 551221769).filter(pl.col("max_videos_in_a_day")== pl.col("max_videos_in_a_day").max())
print(max_videos_per_day_champ1)

#retrieve max # of diff videos in total
max_videos_overall_champ1=max_videos_overall.filter(pl.col("author") == 551221769)
print(max_videos_overall_champ1)

#retrieve number of suspicious day type 1
suspicious_daysf1_champ1=filter_1_days.filter(pl.col("author") == 551221769)
print(suspicious_daysf1_champ1)

    #If in type2: 
suspicious_daysf1_t2_champ1=filter_type2_on_type2.filter(pl.col("author") == 551221769)
print(suspicious_daysf1_t2_champ1)

#retrieve number of suspicious day type 2 
suspicious_daysf2_champ1=filter_2_days.filter(pl.col("author") == 551221769)
print(suspicious_daysf2_champ1)
    #If in type2:
suspicious_daysf2_t2_champ1=numb_app_df.filter(pl.col("author") == 551221769)
print(suspicious_daysf2_t2_champ1)


In [ ]:
# Champions list
champions = [551221769, 260739815, 157898414, 303364156]

# Loop through each champion
for champ in champions:
    print(f"\n=== Retrieving metrics for Champion: {champ} ===")
    
    # Retrieve lifetime
    champ_lifetime = lifetime_df.filter(pl.col("author") == champ)
    print("Lifetime (Overall):")
    print(champ_lifetime)
    
    champ_lifetime_t2 = lifetime_df_type2.filter(pl.col("author") == champ)
    print("Lifetime (Type 2):")
    print(champ_lifetime_t2)
    
    # Retrieve max comments in a day
    row_with_max_daily_comments = df_sus.filter(pl.col("author") == champ).filter(
        pl.col("comments") == pl.col("comments").max()
    )
    print("Max Comments in a Day:")
    print(row_with_max_daily_comments)
    
    # Retrieve total number of comments
    total_com = total_comments_df.filter(pl.col("author") == champ)
    print("Total Comments:")
    print(total_com)
    
    # Retrieve max # of different videos in a day
    max_videos_per_day_champ = max_videos_per_day.filter(pl.col("author") == champ).filter(
        pl.col("max_videos_in_a_day") == pl.col("max_videos_in_a_day").max()
    )
    print("Max Different Videos in a Day:")
    print(max_videos_per_day_champ)
    
    # Retrieve max # of different videos overall
    max_videos_overall = max_videos_overall.filter(pl.col("author") == champ)
    print("Max Different Videos Overall:")
    print(max_videos_overall)
    
    # Retrieve max # of different videos overall in data from type 2
    max_videos_overall_type2=max_videos_overall_type_2.filter(pl.col("author") == champ)
    print("Max Different Videos Overall: (type2)")
    print(max_videos_overall_type2)

    # Retrieve number of suspicious days (Filter 1)
    suspicious_daysf1 = filter_1_days.filter(pl.col("author") == champ)
    print("Suspicious Days (Filter 1):")
    print(suspicious_daysf1)
    
    # Retrieve number of suspicious days (Filter 1, Type 2)
    suspicious_daysf1_t2 = filter_type2_on_type2.filter(pl.col("author") == champ)
    print("Suspicious Days (Filter 1, Type 2):")
    print(suspicious_daysf1_t2)
    
    # Retrieve number of suspicious days (Filter 2)
    suspicious_daysf2 = filter_2_days.filter(pl.col("author") == champ)
    print("Suspicious Days (Filter 2):")
    print(suspicious_daysf2)
    
    # Retrieve number of suspicious days (Filter 2, Type 2)
    suspicious_daysf2_t2 = numb_app_df.filter(pl.col("author") == champ)
    print("Suspicious Days (Filter 2, Type 2):")
    print(suspicious_daysf2_t2)


#### - **Plots**

In [ ]:
# Define the data
champion_metrics = {
    "group": ['C-3PO', 'T-800', 'HAL 9000', 'AGENT SMITH'],
    "LIF": [5245, 4609, 1797, 3396],
    "MVD": [1, 1, 3, 4],
    "MDC": [42, 30990, 6365, 1387],
    "MVT": [4, 31186, 352020, 164069],
    "TLC": [81, 31009, 351984, 154765],
    "NSD_1": [3, 1, 666, 2699],
    "NSD_2": [5, 2, 667, 2907],
}

# Create the Polars DataFrame
champion_metrics_df = pl.DataFrame(champion_metrics)

# Add the "NSD" column as the maximum of "NSD_1" and "NSD_2", then drop the original columns
champion_metrics_df = champion_metrics_df.with_columns(
    pl.max_horizontal(["NSD_1", "NSD_2"]).alias("NSD")  # Compute max horizontally
).drop(["NSD_1", "NSD_2"])  # Drop the original columns

# Convert Polars DataFrame to Pandas DataFrame
champion_metrics_df = champion_metrics_df.to_pandas()

# Normalize only numeric columns and keep the 'group' column intact
champion_metrics_df = champion_metrics_df.set_index("group")  # Set 'group' as the index
champion_metrics_df = champion_metrics_df / champion_metrics_df.max() * 99  # Normalize and scale
champion_metrics_df = champion_metrics_df.round(0)  # Round values
champion_metrics_df = champion_metrics_df.reset_index()  # Restore 'group' as a column


In [ ]:
# ------- PART 1: Define a function that do a plot for one line of the dataset!
 
def make_spider(row, title, color):

    # number of variable
    categories=list(champion_metrics_df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(2,2,row+1, polar=True, )

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=8)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([10,20,30,40,50,60,70,80,90,100], ["10","20","30","40","50","60","70","80","90","100"], color="grey", size=7)
    plt.ylim(0,100)

    # Ind1
    values=champion_metrics_df.loc[row].drop('group').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=11, color=color, y=1.1)

    
# ------- PART 2: Apply the function to all individuals
# initialize the figure
my_dpi=96
plt.figure(figsize=(1000/my_dpi, 1000/my_dpi), dpi=my_dpi)
 
# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))
 
# Loop to plot
for row in range(0, len(champion_metrics_df.index)):
    make_spider( row=row, title=" ", color='red')
plt.savefig('./image_aurel/radar_plts_champions.svg')

In [ ]:
# ------- PART 1: Define a function that creates a radar plot -------
def make_spider(row, title, color, output_path):
    # Number of variables
    categories = list(champion_metrics_df.columns[1:])  # Exclude 'group'
    N = len(categories)

    # Angles for the radar plot
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Close the loop

    # Initialize the radar plot
    fig, ax = plt.subplots(figsize=(5, 5), dpi=96, subplot_kw={"polar": True})

    # Adjust axis directions
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Add labels for axes
    plt.xticks(angles[:-1], categories, color="grey", size=8)

    # Add y-labels
    ax.set_rlabel_position(0)
    plt.yticks(
        [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        ["10", "20", "30", "40", "50", "60", "70", "80", "90", "100"],
        color="grey",
        size=7,
    )
    plt.ylim(0, 100)

    # Data values
    values = champion_metrics_df.iloc[row, 1:].values.flatten().tolist()
    values += values[:1]  # Close the loop

    # Plot the data
    ax.plot(angles, values, color=color, linewidth=2, linestyle="solid")
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=11, color=color, y=1.1)

    # Save the radar plot
    plt.savefig(output_path, bbox_inches="tight")
    plt.close()


# ------- PART 2: Combine radar plot with PNG image -------
def combine_images(radar_path, png_path, output_path):
    # Open both images
    radar_img = Image.open(radar_path)
    png_img = Image.open(png_path)

    # Resize images to have the same height
    png_img = png_img.resize((int(radar_img.width * 0.8), radar_img.height))

    # Create a blank image for the composite
    combined_width = radar_img.width + png_img.width
    combined_img = Image.new("RGB", (combined_width, radar_img.height))

    # Paste the radar and PNG images side by side
    combined_img.paste(png_img, (0, 0))  # Left side: PNG
    combined_img.paste(radar_img, (png_img.width, 0))  # Right side: Radar plot

    # Save the combined image
    combined_img.save(output_path)


# ------- PART 3: Main Loop -------
# Folder containing PNG images
image_folder = "./fut"  # Update with your folder path

# Get a list of all PNG images in the folder
png_files = sorted(
    [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(".png")]
)

champion_metrics_df = pd.DataFrame(champion_metrics)

# Create plots and combine images
output_folder = "./output"
os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

for row, png_path in zip(range(len(champion_metrics_df)), png_files):
    radar_path = os.path.join(output_folder, f"radar_{row + 1}.png")
    output_path = os.path.join(output_folder, f"composite_{row + 1}.png")

    # Create radar plot
    make_spider(row=row, title=f"Group {champion_metrics_df['group'][row]}", color="red", output_path=radar_path)

    # Combine radar plot with the corresponding PNG image
    combine_images(radar_path, png_path, output_path)

    print(f"Created: {output_path}")